# Physics-Informed Neural Networks (PINNs)
## Solving PDEs with Deep Learning

This notebook demonstrates how to implement Physics-Informed Neural Networks (PINNs) for solving partial differential equations.

**Key Idea:** Train a neural network to satisfy:
1. The PDE itself (physics loss)
2. Initial and boundary conditions (data loss)

**Advantages:**
- Mesh-free method
- Can handle high-dimensional problems
- Incorporates physical laws directly into the loss function
- Automatic differentiation provides derivatives

**Example Problem:** 1D Heat Equation
$$\frac{\partial u}{\partial t} = \alpha \frac{\partial^2 u}{\partial x^2}, \quad x \in [0,1], \, t \in [0,T]$$

with boundary conditions: $u(0,t) = u(1,t) = 0$

and initial condition: $u(x,0) = \sin(\pi x)$

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Set device and precision
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)
print(f"Using device: {device}")
print(f"Default dtype: {torch.get_default_dtype()}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## Define the Neural Network Architecture

The network takes $(x, t)$ as input and outputs $u(x,t)$.

We use a fully connected feedforward network with multiple hidden layers and tanh activation.

In [ ]:
class PINN(nn.Module):
    """
    Physics-Informed Neural Network for solving PDEs.
    
    Architecture: Fully connected network with tanh activations
    Input: (x, t) coordinates
    Output: u(x, t) solution
    """
    
    def __init__(self, layers):
        """
        Parameters:
        -----------
        layers : list of int
            Number of neurons in each layer
            Example: [2, 50, 50, 50, 1] for input(x,t) -> 3 hidden layers -> output u
        """
        super(PINN, self).__init__()
        
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
            
    def forward(self, x, t):
        """
        Forward pass through the network.
        
        Parameters:
        -----------
        x : torch.Tensor
            Spatial coordinates
        t : torch.Tensor
            Time coordinates
            
        Returns:
        --------
        u : torch.Tensor
            Network output (solution prediction)
        """
        # Concatenate inputs
        inputs = torch.cat([x, t], dim=1)
        
        # Pass through hidden layers with tanh activation
        out = inputs
        for i in range(len(self.layers) - 1):
            out = torch.tanh(self.layers[i](out))
        
        # Final layer (no activation)
        out = self.layers[-1](out)
        
        return out

# Initialize network
layers = [2, 50, 50, 50, 1]  # 2 inputs (x,t), 3 hidden layers with 50 neurons, 1 output (u)
model = PINN(layers).to(device)

print(f"Network architecture: {layers}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

## Automatic Differentiation for Computing PDE Residuals

PyTorch's autograd allows us to compute derivatives of the network output with respect to inputs.

For the heat equation, we need:
- $\frac{\partial u}{\partial t}$: time derivative
- $\frac{\partial^2 u}{\partial x^2}$: second spatial derivative

In [ ]:
def compute_pde_residual(model, x, t, alpha):
    """
    Compute the PDE residual: du/dt - alpha * d²u/dx²
    
    Parameters:
    -----------
    model : nn.Module
        The PINN model
    x : torch.Tensor
        Spatial coordinates (requires_grad=True)
    t : torch.Tensor
        Time coordinates (requires_grad=True)
    alpha : float
        Thermal diffusivity coefficient
        
    Returns:
    --------
    residual : torch.Tensor
        PDE residual at each point
    """
    # Enable gradient computation
    x.requires_grad_(True)
    t.requires_grad_(True)
    
    # Forward pass
    u = model(x, t)
    
    # Compute first derivatives
    u_t = torch.autograd.grad(
        u, t,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True
    )[0]
    
    u_x = torch.autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True
    )[0]
    
    # Compute second spatial derivative
    u_xx = torch.autograd.grad(
        u_x, x,
        grad_outputs=torch.ones_like(u_x),
        create_graph=True,
        retain_graph=True
    )[0]
    
    # PDE residual: du/dt - alpha * d²u/dx²
    residual = u_t - alpha * u_xx
    
    return residual

## Generate Training Data

We need:
1. **Collocation points**: Random points in the domain for PDE residual
2. **Boundary points**: Points on boundaries $x=0$ and $x=1$
3. **Initial condition points**: Points at $t=0$

In [ ]:
# Domain parameters
x_min, x_max = 0.0, 1.0
t_min, t_max = 0.0, 1.0
alpha = 0.05  # Thermal diffusivity

# Number of training points
N_collocation = 10000  # Interior points for PDE
N_boundary = 100       # Boundary points
N_initial = 100        # Initial condition points

# Collocation points (random points in domain)
x_collocation = torch.rand(N_collocation, 1, device=device) * (x_max - x_min) + x_min
t_collocation = torch.rand(N_collocation, 1, device=device) * (t_max - t_min) + t_min

# Boundary points: x = 0 and x = 1 for all t
t_boundary = torch.rand(N_boundary, 1, device=device) * (t_max - t_min) + t_min
x_boundary_0 = torch.zeros(N_boundary, 1, device=device)
x_boundary_1 = torch.ones(N_boundary, 1, device=device)
u_boundary = torch.zeros(N_boundary, 1, device=device)  # u = 0 at boundaries

# Initial condition points: t = 0 for all x
x_initial = torch.rand(N_initial, 1, device=device) * (x_max - x_min) + x_min
t_initial = torch.zeros(N_initial, 1, device=device)
u_initial = torch.sin(np.pi * x_initial)  # u(x,0) = sin(πx)

print(f"Collocation points: {N_collocation}")
print(f"Boundary points: {2 * N_boundary}")
print(f"Initial condition points: {N_initial}")
print(f"Total training points: {N_collocation + 2*N_boundary + N_initial}")

## Visualize Training Points

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plot collocation points (sample for visibility)
sample_size = 500
ax.scatter(x_collocation[:sample_size].cpu(), t_collocation[:sample_size].cpu(), 
           s=1, c='blue', alpha=0.3, label='Collocation (PDE)')

# Plot boundary points
ax.scatter(x_boundary_0.cpu(), t_boundary.cpu(), 
           s=10, c='red', marker='s', label='Boundary x=0')
ax.scatter(x_boundary_1.cpu(), t_boundary.cpu(), 
           s=10, c='orange', marker='s', label='Boundary x=1')

# Plot initial condition points
ax.scatter(x_initial.cpu(), t_initial.cpu(), 
           s=10, c='green', marker='^', label='Initial condition')

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('t', fontsize=12)
ax.set_title('Training Points Distribution', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Define the Loss Function

The total loss is a weighted sum of:
1. **PDE loss**: $\mathcal{L}_{PDE} = \frac{1}{N_{col}} \sum |\text{PDE residual}|^2$
2. **Boundary loss**: $\mathcal{L}_{BC} = \frac{1}{N_{bc}} \sum |u_{pred} - u_{bc}|^2$
3. **Initial condition loss**: $\mathcal{L}_{IC} = \frac{1}{N_{ic}} \sum |u_{pred} - u_{ic}|^2$

$$\mathcal{L}_{total} = \lambda_{PDE} \mathcal{L}_{PDE} + \lambda_{BC} \mathcal{L}_{BC} + \lambda_{IC} \mathcal{L}_{IC}$$

In [ ]:
def loss_function(model, x_col, t_col, 
                  x_bc0, x_bc1, t_bc, u_bc,
                  x_ic, t_ic, u_ic,
                  alpha, lambda_pde=1.0, lambda_bc=1.0, lambda_ic=1.0):
    """
    Compute the total PINN loss.
    
    Parameters:
    -----------
    model : nn.Module
        The PINN model
    x_col, t_col : torch.Tensor
        Collocation points for PDE residual
    x_bc0, x_bc1, t_bc, u_bc : torch.Tensor
        Boundary condition points and values
    x_ic, t_ic, u_ic : torch.Tensor
        Initial condition points and values
    alpha : float
        PDE parameter (thermal diffusivity)
    lambda_pde, lambda_bc, lambda_ic : float
        Loss weights
        
    Returns:
    --------
    loss : torch.Tensor
        Total loss
    losses_dict : dict
        Dictionary with individual loss components
    """
    
    # 1. PDE residual loss
    residual = compute_pde_residual(model, x_col, t_col, alpha)
    loss_pde = torch.mean(residual ** 2)
    
    # 2. Boundary condition loss
    u_pred_bc0 = model(x_bc0, t_bc)
    u_pred_bc1 = model(x_bc1, t_bc)
    loss_bc = torch.mean((u_pred_bc0 - u_bc) ** 2) + torch.mean((u_pred_bc1 - u_bc) ** 2)
    
    # 3. Initial condition loss
    u_pred_ic = model(x_ic, t_ic)
    loss_ic = torch.mean((u_pred_ic - u_ic) ** 2)
    
    # Total loss
    loss = lambda_pde * loss_pde + lambda_bc * loss_bc + lambda_ic * loss_ic
    
    losses_dict = {
        'total': loss.item(),
        'pde': loss_pde.item(),
        'bc': loss_bc.item(),
        'ic': loss_ic.item()
    }
    
    return loss, losses_dict

## Training Loop

In [ ]:
# Training parameters
learning_rate = 1e-3
num_epochs = 5000
print_every = 500

# Loss weights
lambda_pde = 1.0
lambda_bc = 100.0  # Higher weight for boundary conditions
lambda_ic = 100.0  # Higher weight for initial conditions

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=500, verbose=True
)

# Training history
history = {
    'total': [],
    'pde': [],
    'bc': [],
    'ic': []
}

print("Starting training...")
print(f"Epochs: {num_epochs}")
print(f"Learning rate: {learning_rate}")
print(f"Loss weights - PDE: {lambda_pde}, BC: {lambda_bc}, IC: {lambda_ic}")
print()

import time
start_time = time.time()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Compute loss
    loss, losses_dict = loss_function(
        model, x_collocation, t_collocation,
        x_boundary_0, x_boundary_1, t_boundary, u_boundary,
        x_initial, t_initial, u_initial,
        alpha, lambda_pde, lambda_bc, lambda_ic
    )
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Update scheduler
    scheduler.step(loss)
    
    # Store history
    for key in history.keys():
        history[key].append(losses_dict[key])
    
    # Print progress
    if (epoch + 1) % print_every == 0:
        elapsed = time.time() - start_time
        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Total: {losses_dict['total']:.6e} | "
              f"PDE: {losses_dict['pde']:.6e} | "
              f"BC: {losses_dict['bc']:.6e} | "
              f"IC: {losses_dict['ic']:.6e} | "
              f"Time: {elapsed:.2f}s")

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time:.2f} seconds")
print(f"Final loss: {history['total'][-1]:.6e}")

## Plot Training History

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot total loss
axes[0].semilogy(history['total'], 'b-', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Total Loss', fontsize=12)
axes[0].set_title('Total Loss vs Epoch', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Plot individual losses
axes[1].semilogy(history['pde'], label='PDE', linewidth=2)
axes[1].semilogy(history['bc'], label='Boundary', linewidth=2)
axes[1].semilogy(history['ic'], label='Initial', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss Component', fontsize=12)
axes[1].set_title('Loss Components vs Epoch', fontsize=14)
axes[1].legend(fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Evaluate the Trained Model

Create a fine grid and evaluate the PINN solution.

In [ ]:
# Create evaluation grid
n_x = 100
n_t = 100

x_eval = np.linspace(x_min, x_max, n_x)
t_eval = np.linspace(t_min, t_max, n_t)
X_eval, T_eval = np.meshgrid(x_eval, t_eval)

# Flatten for network input
x_test = torch.tensor(X_eval.flatten()[:, None], device=device)
t_test = torch.tensor(T_eval.flatten()[:, None], device=device)

# Evaluate model
model.eval()
with torch.no_grad():
    u_pred = model(x_test, t_test).cpu().numpy()

U_pred = u_pred.reshape(n_t, n_x)

print(f"Evaluation grid: {n_x} x {n_t} points")
print(f"Solution range: [{U_pred.min():.6f}, {U_pred.max():.6f}]")

## Analytical Solution

For the given problem, the analytical solution is:
$$u(x,t) = e^{-\alpha \pi^2 t} \sin(\pi x)$$

In [ ]:
# Compute analytical solution
U_exact = np.exp(-alpha * np.pi**2 * T_eval) * np.sin(np.pi * X_eval)

# Compute error
error = np.abs(U_pred - U_exact)
relative_error = np.linalg.norm(U_pred - U_exact) / np.linalg.norm(U_exact)

print(f"L2 relative error: {relative_error:.6e}")
print(f"Max absolute error: {error.max():.6e}")
print(f"Mean absolute error: {error.mean():.6e}")

## Visualize Results

In [ ]:
fig = plt.figure(figsize=(18, 5))

# PINN solution
ax1 = fig.add_subplot(131, projection='3d')
surf1 = ax1.plot_surface(X_eval, T_eval, U_pred, cmap='viridis', alpha=0.9)
ax1.set_xlabel('x', fontsize=10)
ax1.set_ylabel('t', fontsize=10)
ax1.set_zlabel('u(x,t)', fontsize=10)
ax1.set_title('PINN Solution', fontsize=12)
fig.colorbar(surf1, ax=ax1, shrink=0.5)

# Analytical solution
ax2 = fig.add_subplot(132, projection='3d')
surf2 = ax2.plot_surface(X_eval, T_eval, U_exact, cmap='viridis', alpha=0.9)
ax2.set_xlabel('x', fontsize=10)
ax2.set_ylabel('t', fontsize=10)
ax2.set_zlabel('u(x,t)', fontsize=10)
ax2.set_title('Analytical Solution', fontsize=12)
fig.colorbar(surf2, ax=ax2, shrink=0.5)

# Error
ax3 = fig.add_subplot(133, projection='3d')
surf3 = ax3.plot_surface(X_eval, T_eval, error, cmap='hot', alpha=0.9)
ax3.set_xlabel('x', fontsize=10)
ax3.set_ylabel('t', fontsize=10)
ax3.set_zlabel('|error|', fontsize=10)
ax3.set_title('Absolute Error', fontsize=12)
fig.colorbar(surf3, ax=ax3, shrink=0.5)

plt.tight_layout()
plt.show()

## 2D Comparison: Contour Plots

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# PINN solution
im1 = axes[0].contourf(X_eval, T_eval, U_pred, levels=20, cmap='viridis')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('t', fontsize=12)
axes[0].set_title('PINN Solution', fontsize=14)
fig.colorbar(im1, ax=axes[0])

# Analytical solution
im2 = axes[1].contourf(X_eval, T_eval, U_exact, levels=20, cmap='viridis')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('t', fontsize=12)
axes[1].set_title('Analytical Solution', fontsize=14)
fig.colorbar(im2, ax=axes[1])

# Error
im3 = axes[2].contourf(X_eval, T_eval, error, levels=20, cmap='hot')
axes[2].set_xlabel('x', fontsize=12)
axes[2].set_ylabel('t', fontsize=12)
axes[2].set_title('Absolute Error', fontsize=14)
fig.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

## Time Slices Comparison

In [ ]:
# Plot solution at different time points
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

time_indices = [0, 19, 39, 59, 79, 99]

for idx, t_idx in enumerate(time_indices):
    ax = axes[idx]
    t = t_eval[t_idx]
    
    ax.plot(x_eval, U_pred[t_idx, :], 'b-', linewidth=2, label='PINN')
    ax.plot(x_eval, U_exact[t_idx, :], 'r--', linewidth=2, label='Analytical')
    ax.set_xlabel('x', fontsize=10)
    ax.set_ylabel('u(x,t)', fontsize=10)
    ax.set_title(f't = {t:.3f}', fontsize=12)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_ylim([-0.1, 1.1])

plt.tight_layout()
plt.show()

## Check PDE Residual on Test Points

Evaluate how well the trained network satisfies the PDE on unseen points.

In [ ]:
# Generate test points
N_test = 1000
x_test_pde = torch.rand(N_test, 1, device=device) * (x_max - x_min) + x_min
t_test_pde = torch.rand(N_test, 1, device=device) * (t_max - t_min) + t_min

# Compute residual
model.eval()
residual_test = compute_pde_residual(model, x_test_pde, t_test_pde, alpha)
residual_test_np = residual_test.detach().cpu().numpy()

print(f"PDE residual statistics on {N_test} test points:")
print(f"  Mean: {residual_test_np.mean():.6e}")
print(f"  Std: {residual_test_np.std():.6e}")
print(f"  Max absolute: {np.abs(residual_test_np).max():.6e}")
print(f"  RMS: {np.sqrt(np.mean(residual_test_np**2)):.6e}")

# Plot residual distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(residual_test_np, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('PDE Residual', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of PDE Residuals', fontsize=14)
axes[0].grid(True, alpha=0.3)

scatter = axes[1].scatter(x_test_pde.cpu(), t_test_pde.cpu(), 
                          c=np.abs(residual_test_np), s=20, cmap='hot')
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('t', fontsize=12)
axes[1].set_title('Absolute PDE Residual in Domain', fontsize=14)
fig.colorbar(scatter, ax=axes[1], label='|Residual|')

plt.tight_layout()
plt.show()

## Summary and Next Steps

### What we accomplished:
1. ✅ Built a PINN architecture for solving PDEs
2. ✅ Used automatic differentiation to compute PDE residuals
3. ✅ Combined physics loss with data loss (BCs and ICs)
4. ✅ Trained the network successfully
5. ✅ Validated against analytical solution

### Advantages of PINNs:
- Mesh-free: No need to discretize the domain explicitly
- Flexible: Can handle complex geometries and irregular domains
- Incorporates physics: The PDE is enforced during training
- Differentiable: Can compute derivatives analytically via autograd

### Limitations and challenges:
- Training can be slow and may require careful tuning
- Loss balancing between different terms is crucial
- May struggle with sharp gradients or discontinuities
- No guarantee of convergence to the correct solution

### Extensions:
1. **Harder PDEs**: Navier-Stokes, wave equation, etc.
2. **Inverse problems**: Learn unknown PDE parameters from data
3. **Multi-fidelity**: Combine low and high-fidelity data
4. **Adaptive sampling**: Focus training points where error is large
5. **Neural operators**: Learn solution operators instead of individual solutions
6. **Higher dimensions**: 2D, 3D spatial domains
7. **Complex geometries**: Non-rectangular domains with curved boundaries